In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
#Loading the json file with intents
# import our chat-bot intents file
import json
with open('C:/ChatBot/Tensorflow/intents.json') as json_data:
    intents = json.load(json_data)

## Organizing our documents, words and classification classes.

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']

In [6]:
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [29]:
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

In [8]:
intents

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['Which mopeds do you have?',
    'What kinds of mopeds are there?',
    'What do you rent?'],
   'responses': ['We rent Yamaha, Piaggio and Vespa mopeds',
    'We have Piaggio, Vespa and Yamaha mopeds'

In [9]:
intent["tag"][0]

't'

In [7]:
classes

['greeting',
 'goodbye',
 'thanks',
 'hours',
 'mopeds',
 'payments',
 'opentoday',
 'rental',
 'today']

In [7]:
# remove duplicates
classes = sorted(list(set(classes)))

In [8]:
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [22]:
classes

['goodbye',
 'greeting',
 'hours',
 'mopeds',
 'opentoday',
 'payments',
 'rental',
 'thanks',
 'today']

## This data structure won’t work with Tensorflow, we need to transform it further: from documents of words into tensors of numbers.

In [23]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [24]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [20]:
words

["'d",
 "'s",
 'a',
 'acceiv',
 'anyon',
 'ar',
 'bye',
 'can',
 'card',
 'cash',
 'credit',
 'day',
 'do',
 'doe',
 'good',
 'goodby',
 'hav',
 'hello',
 'help',
 'hi',
 'hour',
 'how',
 'i',
 'is',
 'kind',
 'lat',
 'lik',
 'mastercard',
 'mop',
 'of',
 'on',
 'op',
 'rent',
 'see',
 'tak',
 'thank',
 'that',
 'ther',
 'thi',
 'to',
 'today',
 'we',
 'what',
 'when',
 'which',
 'work',
 'yo',
 'you']

In [21]:
classes

['goodbye',
 'greeting',
 'hours',
 'mopeds',
 'opentoday',
 'payments',
 'rental',
 'thanks',
 'today']

In [11]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

## Our data is shuffled. Tensorflow will take some of this and use it as test data to gauge accuracy for a newly fitted model.

In [15]:
training[:,1]

array([list([0, 0, 0, 0, 0, 0, 0, 1, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 1, 0]),
       list([0, 1, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 0, 1]),
       list([0, 1, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 1, 0, 0]),
       list([1, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 1, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 1, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 1, 0, 0, 0, 0]),
       list([0, 1, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 1, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 0, 1, 0]),
       list([0, 0, 0, 0, 1, 0, 0, 0, 0]),
       list([0, 0, 1, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 1, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 1, 0, 0, 0]),
       list([0, 1, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 0, 0, 1, 0, 0]),
       list([0, 0, 1, 0, 0, 0, 0, 0, 0]),
       list([0, 1, 0, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 1, 0, 0, 0, 0, 0, 0]),
       list([0, 0, 0, 0, 1, 0, 0, 0, 0]),
       list([1, 0, 0, 0, 0, 0, 0, 

In [13]:
train_y

[[0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0]]

In [12]:
# Building our model
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

In [13]:
# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('C:/ChatBot/Tensorflow/model.tflearn')

Training Step: 3999  | total loss: 0.14488 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.14488 - acc: 0.9988 -- iter: 24/27
Training Step: 4000  | total loss: 0.13696 | time: 0.020s
| Adam | epoch: 1000 | loss: 0.13696 - acc: 0.9989 -- iter: 27/27
--
INFO:tensorflow:C:/ChatBot/Tensorflow/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [16]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [17]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [18]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [19]:
print(model.predict([p]))

[[4.629927730093186e-07, 0.0004036641039419919, 0.12859593331813812, 2.529509117721318e-07, 0.8209863901138306, 0.010348940268158913, 3.8250490774771606e-08, 3.1558582122670487e-05, 0.03963283821940422]]


## We’ll save (‘pickle’) our model and documents for future use

In [20]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "C:/ChatBot/Tensorflow/training_data", "wb" ) )

In [17]:
type(intents)

dict